# Intent Classification

In [68]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [26]:
json = {"intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
     "responses": ["Hello", "Hi", "Hi there"]
    },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye"],
     "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
     "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
    },
    {"tag": "about",
     "patterns": ["Who are you?", "What are you?", "Who you are?" ],
     "responses": ["I.m Joana, your bot assistant", "I'm Joana, an Artificial Intelligent bot"]
    },
    {"tag": "name",
    "patterns": ["what is your name", "what should I call you", "whats your name?"],
    "responses": ["You can call me Joana.", "I'm Joana!", "Just call me as Joana"]
    },
    {"tag": "help",
    "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
    "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
    },
    {"tag": "createaccount",
    "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
    "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
    },
    {"tag": "complaint",
    "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
    "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
    }
]
}

In [27]:
patterns = []

In [28]:
for intent in json["intents"]:
  for pattern in intent["patterns"]:
    if pattern not in patterns:
      patterns.append(pattern)

In [29]:
patterns[:5]

['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay']

In [30]:
dict_ = {}

In [31]:
for pattern in patterns:
  for intent in json["intents"]:
    if pattern in intent["patterns"]:
      dict_[pattern] = intent["tag"]

In [32]:
for k, v in dict_.items():
  print(k, ":", v)

Hi : greeting
Hey : greeting
Is anyone there? : greeting
Hello : greeting
Hay : greeting
Bye : goodbye
See you later : goodbye
Goodbye : goodbye
Thanks : thanks
Thank you : thanks
That's helpful : thanks
Thanks for the help : thanks
Who are you? : about
What are you? : about
Who you are? : about
what is your name : name
what should I call you : name
whats your name? : name
Could you help me? : help
give me a hand please : help
Can you help? : help
What can you do for me? : help
I need a support : help
I need a help : help
support me please : help
I need to create a new account : createaccount
how to open a new account : createaccount
I want to create an account : createaccount
can you create an account for me : createaccount
have a complaint : complaint
I want to raise a complaint : complaint
there is a complaint about a service : complaint


In [33]:
dict2={}

In [34]:
for intent in json["intents"]:
  dict2[intent["tag"]] = intent["responses"]

In [40]:
dict2

{'about': ['I.m Joana, your bot assistant',
  "I'm Joana, an Artificial Intelligent bot"],
 'complaint': ['Please provide us your complaint in order to assist you',
  'Please mention your complaint, we will reach you and sorry for any inconvenience caused'],
 'createaccount': ['You can just easily create a new account from our web site',
  'Just go to our web site and follow the guidelines to create a new account'],
 'goodbye': ['See you later', 'Have a nice day', 'Bye! Come back again'],
 'greeting': ['Hello', 'Hi', 'Hi there'],
 'help': ['Tell me how can assist you',
  'Tell me your problem to assist you',
  'Yes Sure, How can I support you'],
 'name': ['You can call me Joana.', "I'm Joana!", 'Just call me as Joana'],
 'thanks': ['Happy to help!',
  'Any time!',
  'My pleasure',
  "You're most welcome!"]}

In [41]:
#Unique classes
classes = [*dict2.keys()]
classes

['greeting',
 'goodbye',
 'thanks',
 'about',
 'name',
 'help',
 'createaccount',
 'complaint']

In [42]:
for k, v in dict2.items():
  print(k, ":", v)

greeting : ['Hello', 'Hi', 'Hi there']
goodbye : ['See you later', 'Have a nice day', 'Bye! Come back again']
thanks : ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"]
about : ['I.m Joana, your bot assistant', "I'm Joana, an Artificial Intelligent bot"]
name : ['You can call me Joana.', "I'm Joana!", 'Just call me as Joana']
help : ['Tell me how can assist you', 'Tell me your problem to assist you', 'Yes Sure, How can I support you']
createaccount : ['You can just easily create a new account from our web site', 'Just go to our web site and follow the guidelines to create a new account']
complaint : ['Please provide us your complaint in order to assist you', 'Please mention your complaint, we will reach you and sorry for any inconvenience caused']


In [43]:
labels = []

In [44]:
for v in dict_.values():
  labels.append(classes.index(v))

In [45]:
len(labels)

32

In [47]:
#Our corpus/corpus of documents
list_of_documents = list(dict_.keys())
list_of_documents

['Hi',
 'Hey',
 'Is anyone there?',
 'Hello',
 'Hay',
 'Bye',
 'See you later',
 'Goodbye',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Thanks for the help',
 'Who are you?',
 'What are you?',
 'Who you are?',
 'what is your name',
 'what should I call you',
 'whats your name?',
 'Could you help me?',
 'give me a hand please',
 'Can you help?',
 'What can you do for me?',
 'I need a support',
 'I need a help',
 'support me please',
 'I need to create a new account',
 'how to open a new account',
 'I want to create an account',
 'can you create an account for me',
 'have a complaint',
 'I want to raise a complaint',
 'there is a complaint about a service']

In [49]:
len(list_of_documents)

32

In [50]:
vectorizer = CountVectorizer()
vectorizer.fit(list_of_documents)
vector = vectorizer.transform(list_of_documents)

In [51]:
vectorizer.vocabulary_

{'about': 0,
 'account': 1,
 'an': 2,
 'anyone': 3,
 'are': 4,
 'bye': 5,
 'call': 6,
 'can': 7,
 'complaint': 8,
 'could': 9,
 'create': 10,
 'do': 11,
 'for': 12,
 'give': 13,
 'goodbye': 14,
 'hand': 15,
 'have': 16,
 'hay': 17,
 'hello': 18,
 'help': 19,
 'helpful': 20,
 'hey': 21,
 'hi': 22,
 'how': 23,
 'is': 24,
 'later': 25,
 'me': 26,
 'name': 27,
 'need': 28,
 'new': 29,
 'open': 30,
 'please': 31,
 'raise': 32,
 'see': 33,
 'service': 34,
 'should': 35,
 'support': 36,
 'thank': 37,
 'thanks': 38,
 'that': 39,
 'the': 40,
 'there': 41,
 'to': 42,
 'want': 43,
 'what': 44,
 'whats': 45,
 'who': 46,
 'you': 47,
 'your': 48}

In [52]:
X = vector.toarray()
len(X[0])

49

In [53]:
X[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [56]:
svc = LinearSVC(random_state=69, dual=False)
svc.fit(X, labels)

LinearSVC(dual=False, random_state=69)

In [57]:
print(f'LinearSVC test accuracy: {svc.score(X, labels)*100:.2f}%')

LinearSVC test accuracy: 100.00%


In [59]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list_of_documents)
word_index = tokenizer.word_index
print(f"Found {len(word_index)} unique tokens")

Found 51 unique tokens


In [ ]:
word_index

{'a': 2,
 'about': 50,
 'account': 8,
 'an': 25,
 'anyone': 28,
 'are': 11,
 'bye': 31,
 'call': 40,
 'can': 12,
 'complaint': 15,
 'could': 42,
 'create': 14,
 'do': 45,
 'for': 10,
 'give': 43,
 'goodbye': 34,
 'hand': 44,
 'have': 48,
 'hay': 30,
 'hello': 29,
 'help': 5,
 'helpful': 37,
 'hey': 27,
 'hi': 26,
 'how': 46,
 'i': 3,
 'is': 9,
 'later': 33,
 'me': 4,
 'name': 20,
 'need': 13,
 'new': 23,
 'open': 47,
 'please': 21,
 'raise': 49,
 'see': 32,
 'service': 51,
 'should': 39,
 'support': 22,
 'thank': 35,
 'thanks': 17,
 "that's": 36,
 'the': 38,
 'there': 16,
 'to': 7,
 'want': 24,
 'what': 6,
 'whats': 41,
 'who': 18,
 'you': 1,
 'your': 19}

In [60]:
X_ = tokenizer.texts_to_sequences(list_of_documents)
X_

[[26],
 [27],
 [9, 28, 16],
 [29],
 [30],
 [31],
 [32, 1, 33],
 [34],
 [17],
 [35, 1],
 [36, 37],
 [17, 10, 38, 5],
 [18, 11, 1],
 [6, 11, 1],
 [18, 1, 11],
 [6, 9, 19, 20],
 [6, 39, 3, 40, 1],
 [41, 19, 20],
 [42, 1, 5, 4],
 [43, 4, 2, 44, 21],
 [12, 1, 5],
 [6, 12, 1, 45, 10, 4],
 [3, 13, 2, 22],
 [3, 13, 2, 5],
 [22, 4, 21],
 [3, 13, 7, 14, 2, 23, 8],
 [46, 7, 47, 2, 23, 8],
 [3, 24, 7, 14, 25, 8],
 [12, 1, 14, 25, 8, 10, 4],
 [48, 2, 15],
 [3, 24, 7, 49, 2, 15],
 [16, 9, 2, 15, 50, 2, 51]]

In [61]:
MAX_SEQUENCE_LENGTH = max(len(encoded_document) for encoded_document in X_)
MAX_SEQUENCE_LENGTH

7

In [62]:
X_ = pad_sequences(X_, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post")
print("Shape of data tensor:", X_.shape)

Shape of data tensor: (32, 7)


In [63]:
X_

array([[26,  0,  0,  0,  0,  0,  0],
       [27,  0,  0,  0,  0,  0,  0],
       [ 9, 28, 16,  0,  0,  0,  0],
       [29,  0,  0,  0,  0,  0,  0],
       [30,  0,  0,  0,  0,  0,  0],
       [31,  0,  0,  0,  0,  0,  0],
       [32,  1, 33,  0,  0,  0,  0],
       [34,  0,  0,  0,  0,  0,  0],
       [17,  0,  0,  0,  0,  0,  0],
       [35,  1,  0,  0,  0,  0,  0],
       [36, 37,  0,  0,  0,  0,  0],
       [17, 10, 38,  5,  0,  0,  0],
       [18, 11,  1,  0,  0,  0,  0],
       [ 6, 11,  1,  0,  0,  0,  0],
       [18,  1, 11,  0,  0,  0,  0],
       [ 6,  9, 19, 20,  0,  0,  0],
       [ 6, 39,  3, 40,  1,  0,  0],
       [41, 19, 20,  0,  0,  0,  0],
       [42,  1,  5,  4,  0,  0,  0],
       [43,  4,  2, 44, 21,  0,  0],
       [12,  1,  5,  0,  0,  0,  0],
       [ 6, 12,  1, 45, 10,  4,  0],
       [ 3, 13,  2, 22,  0,  0,  0],
       [ 3, 13,  2,  5,  0,  0,  0],
       [22,  4, 21,  0,  0,  0,  0],
       [ 3, 13,  7, 14,  2, 23,  8],
       [46,  7, 47,  2, 23,  8,  0],
 

In [65]:
Y = np.array(labels)

In [66]:
Y.shape

(32,)

In [67]:
X_.shape

(32, 7)

In [70]:
n_classes = len(classes)
n_classes

8

In [74]:
EMBEDDING_DIM = 32
MAX_NB_WORDS = len(word_index) + 1

In [77]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_.shape[1]))
model.add(tf.keras.layers.SpatialDropout1D(.1))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=.2)))
model.add(tf.keras.layers.Dense(n_classes, activation="softmax"))
opt = tf.keras.optimizers.Adam(learning_rate=.0003)
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer=opt, metrics=["accuracy"])

epochs = 75
batch_size = 4

history = model.fit(X_, Y, epochs=epochs, batch_size=batch_size, validation_split = 0.1)

Epoch 1/75
7/7 [==============================] - 4s 151ms/step - loss: 2.0776 - accuracy: 0.2143 - val_loss: 2.0810 - val_accuracy: 0.2500
Epoch 2/75
7/7 [==============================] - 0s 10ms/step - loss: 2.0693 - accuracy: 0.4286 - val_loss: 2.0887 - val_accuracy: 0.2500
Epoch 3/75
7/7 [==============================] - 0s 9ms/step - loss: 2.0625 - accuracy: 0.3571 - val_loss: 2.0968 - val_accuracy: 0.2500
Epoch 4/75
7/7 [==============================] - 0s 10ms/step - loss: 2.0576 - accuracy: 0.3214 - val_loss: 2.1052 - val_accuracy: 0.0000e+00
Epoch 5/75
7/7 [==============================] - 0s 9ms/step - loss: 2.0509 - accuracy: 0.3929 - val_loss: 2.1143 - val_accuracy: 0.0000e+00
Epoch 6/75
7/7 [==============================] - 0s 9ms/step - loss: 2.0446 - accuracy: 0.2500 - val_loss: 2.1241 - val_accuracy: 0.0000e+00
Epoch 7/75
7/7 [==============================] - 0s 9ms/step - loss: 2.0336 - accuracy: 0.3214 - val_loss: 2.1353 - val_accuracy: 0.0000e+00
Epoch 8/75
7/7

In [91]:
text = input("Unesite upit: ")
document = [text]
x = tokenizer.texts_to_sequences(document)
x = pad_sequences(x, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post")
pred = tf.math.argmax(model.predict(x)[0]).numpy()
print("intent:", classes[pred])

Unesite upit: how can I help you
intent: help
